# Notebook 1 – Data Cleaning & Feature Engineering
**Project:** Judicial Vacancy → Nomination/Confirmation Pipeline

*Initial draft generated via ChatGPT model o3 on 2025-07-12T02:40:38.399372Z*

In [ ]:

import sys
from pathlib import Path

import pandas as pd
from loguru import logger

# Add the project root to the path so we can import our modules
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))


# Setup logging
logger.remove()  # Remove default handler
logger.add(sys.stderr, format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level}</level> | <cyan>{function}</cyan> - <level>{message}</level>", level="INFO")

1

# Load dataframes from Raw data folder

Start with loading simpler, non-JSON-containing CSV files

In [ ]:
from nomination_predictor.config import INTERIM_DATA_DIR, RAW_DATA_DIR

# load FJC dataframes (and derived seat timeline)
fjc_judges = pd.read_csv(RAW_DATA_DIR / "judges.csv")
fjc_federal_judicial_service = pd.read_csv(RAW_DATA_DIR / "federal_judicial_service.csv")
fjc_demographics = pd.read_csv(RAW_DATA_DIR / "demographics.csv")
fjc_education = pd.read_csv(RAW_DATA_DIR / "education.csv")
fjc_other_federal_judicial_service = pd.read_csv(
    RAW_DATA_DIR / "other_federal_judicial_service.csv"
)
fjc_other_nominations_recess = pd.read_csv(RAW_DATA_DIR / "other_nominations_recess.csv")
fjc_professional_career = pd.read_csv(RAW_DATA_DIR / "professional_career.csv")
#seat_timeline = pd.read_csv(RAW_DATA_DIR / "seat_timeline.csv")

2025-07-15 19:32:52.378 | INFO     | nomination_predictor.config:<module>:103 - Project root: /home/wsl2ubuntuuser/nomination_predictor
2025-07-15 19:32:52.382 | INFO     | nomination_predictor.config:<module>:127 - Configuration loaded


## Combine all dataframes into a single dictionary for bulk operations

In [ ]:
# Combine all dataframes into a single dictionary for bulk operations


# Start with FJC dataframes; we'll add Congress ones soon
dfs = {
    # FJC dataframes
    "fjc_judges": fjc_judges,
    "fjc_federal_judicial_service": fjc_federal_judicial_service,
    "fjc_demographics": fjc_demographics,
    "fjc_education": fjc_education,
    "fjc_other_federal_judicial_service": fjc_other_federal_judicial_service,
    "fjc_other_nominations_recess": fjc_other_nominations_recess,
    "fjc_professional_career": fjc_professional_career,
    #"seat_timeline": seat_timeline,
    
    # Congress dataframes
}

## Load & immediately drop duplicated rows from congress API data

If we made the same request multiple times and got the same response (e.g. from a software design oversight, or pausing & resuming/mashing together downloads made on separate occasions with possible overlap)

It's easier to find this kind of duplicate now vs. after flattening.

In [ ]:
from nomination_predictor.features import flatten_json_dataframe

# Load Congress API dataframes
cong_nominations_raw = pd.read_csv(RAW_DATA_DIR / "nominations.csv")
cong_nominees_raw = pd.read_csv(RAW_DATA_DIR / "nominees.csv")

dfs["cong_nominations"] = cong_nominations_raw
dfs["cong_nominees"] = cong_nominees_raw

In [ ]:
# ------------------------------------------------------------------
# 1. Identify the rows that *would* be dropped
dupe_mask = dfs["cong_nominations"].duplicated(subset=["nomination", "request"], keep="first")
dupes      = dfs["cong_nominations"].loc[dupe_mask].copy()

# ------------------------------------------------------------------
# 2. Show a compact summary
print(f"Rows flagged as duplicates: {len(dupes)}")
display(
    dupes.sort_values(["nomination", "request"])
         .head(20)   # show first 20; remove .head() to see all
)

# Optional: see how many duplicates per citation
dup_counts = (
    dfs["cong_nominations"]
      .loc[dupe_mask, "nomination"]
      .value_counts()
      .head(10)
)
print("\nTop duplicate records:")
display(dup_counts)

Rows flagged as duplicates: 189


nomination  \
4956  {'actions': {'count': 1, 'url': 'https://api.c...   
4953  {'actions': {'count': 1, 'url': 'https://api.c...   
4725  {'actions': {'count': 1, 'url': 'https://api.c...   
4494  {'actions': {'count': 1, 'url': 'https://api.c...   
4495  {'actions': {'count': 1, 'url': 'https://api.c...   
1675  {'actions': {'count': 11, 'url': 'https://api....   
1676  {'actions': {'count': 11, 'url': 'https://api....   
1146  {'actions': {'count': 11, 'url': 'https://api....   
20    {'actions': {'count': 11, 'url': 'https://api....   
140   {'actions': {'count': 11, 'url': 'https://api....   
1640  {'actions': {'count': 12, 'url': 'https://api....   
1627  {'actions': {'count': 12, 'url': 'https://api....   
1628  {'actions': {'count': 12, 'url': 'https://api....   
1563  {'actions': {'count': 12, 'url': 'https://api....   
1566  {'actions': {'count': 12, 'url': 'https://api....   
122   {'actions': {'count': 12, 'url': 'https://api....   
1588  {'actions': {'count': 13, 'url': 'https://api....   
1571  {'actions': {'count': 13, 'url': 'https://api....   
1528  {'actions': {'count': 13, 'url': 'https://api....   
141   {'actions': {'count': 13, 'url': 'https://api....   

                                                request retrieval_date  \
4956  {'congress': '100', 'contentType': 'applicatio...     2025-07-12   
4953  {'congress': '100', 'contentType': 'applicatio...     2025-07-12   
4725  {'congress': '101', 'contentType': 'applicatio...     2025-07-12   
4494  {'congress': '102', 'contentType': 'applicatio...     2025-07-12   
4495  {'congress': '102', 'contentType': 'applicatio...     2025-07-12   
1675  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
1676  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
1146  {'congress': '115', 'contentType': 'applicatio...     2025-07-12   
20    {'congress': '118', 'contentType': 'applicatio...     2025-07-12   
140   {'congress': '118', 'contentType': 'applicatio...     2025-07-12   
1640  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
1627  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
1628  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
1563  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
1566  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
122   {'congress': '118', 'contentType': 'applicatio...     2025-07-12   
1588  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
1571  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
1528  {'congress': '113', 'contentType': 'applicatio...     2025-07-12   
141   {'congress': '118', 'contentType': 'applicatio...     2025-07-12   

      is_full_detail  
4956            True  
4953            True  
4725            True  
4494            True  
4495            True  
1675            True  
1676            True  
1146            True  
20              True  
140             True  
1640            True  
1627            True  
1628            True  
1563            True  
1566            True  
122             True  
1588            True  
1571            True  
1528            True  
141             True


Top duplicate records:


nomination
{'actions': {'count': 2, 'url': 'https://api.congress.gov/v3/nomination/101/1181-0/actions?format=json'}, 'authorityDate': '1989-01-03', 'citation': 'PN1181-0', 'committees': {'count': 1, 'url': 'https://api.congress.gov/v3/nomination/101/1181-0/committees?format=json'}, 'congress': 101, 'description': 'The following-named persons to be Associate Judges of the Superior Court of the District of Columbia for the term of fifteen years. (New Positions) \tMary Ellen Abrecht, of the District of Columbia. \tKaye K. Christian, of the District of Columbia. \tFrederick D. Dorsey, of the District of Columbia. \tEllen Segal Huvelle, of the District of Columbia. \tJose M. Lopez, of the District of Columbia. \tJoan Z. McAvoy, of the District of Columbia. \tGregory E. Mize, of the District of Columbia. \tPatricia A. Wynn, of the District of Columbia.', 'latestAction': {'actionDate': '1990-08-02', 'text': 'Committee on Governmental Affairs. Ordered to be reported favorably.'}, 'nominationTy

Then we can remove the duplicates, keeping the one with the most recent retrieval_date on the presumption it's the one most likely to have been corrected for accuracy server-side.

In [ ]:
print(f"shape before checking for & dropping duplicated congressional nomination records is { dfs["cong_nominations"].shape}")

# First sort by retrieval_date in ascending order
# Then drop duplicates keeping the last occurrence (which will be the most recent date)
dfs["cong_nominations"] = (dfs["cong_nominations"]
                          .sort_values("retrieval_date")
                          .drop_duplicates(subset=["nomination", "request"], keep='last'))

print(f"shape after checking for & dropping duplicated congressional nomination records is { dfs["cong_nominations"].shape}")

shape before checking for & dropping duplicated congressional nomination records is (5746, 4)
shape after checking for & dropping duplicated congressional nomination records is (5557, 4)


Same logic for nominee dataframe, target column just has a different name

TODO: if this operation works well, refactor it to features.py taking dataframe and target column names as inputs

In [ ]:
# ------------------------------------------------------------------
# 1. Identify the rows that *would* be dropped
dupe_mask = dfs["cong_nominees"].duplicated(subset=["nominee", "request"], keep="first")
dupes      = dfs["cong_nominees"].loc[dupe_mask].copy()

# ------------------------------------------------------------------
# 2. Show a compact summary
print(f"Rows flagged as duplicates: {len(dupes)}")
display(
    dupes.sort_values(["nominee", "request"])
         .head(20)   # show first 20; remove .head() to see all
)

# Optional: see how many duplicates per citation
dup_counts = (
    dfs["cong_nominees"]
      .loc[dupe_mask, "nominee"]
      .value_counts()
      .head(10)
)
print("\nTop duplicate records:")
display(dup_counts)

Rows flagged as duplicates: 155


nominee  \
119   {'nominees': [{'firstName': 'Adam', 'lastName'...   
778   {'nominees': [{'firstName': 'Adam', 'lastName'...   
336   {'nominees': [{'firstName': 'Almo', 'lastName'...   
492   {'nominees': [{'firstName': 'Andrew', 'lastNam...   
1562  {'nominees': [{'firstName': 'Arthur', 'lastNam...   
111   {'nominees': [{'firstName': 'Benjamin', 'lastN...   
1022  {'nominees': [{'firstName': 'Bradley', 'lastNa...   
1106  {'nominees': [{'firstName': 'Bradley', 'lastNa...   
1108  {'nominees': [{'firstName': 'Brendan', 'lastNa...   
4914  {'nominees': [{'firstName': 'Bruce', 'lastName...   
78    {'nominees': [{'firstName': 'Caroline', 'lastN...   
1034  {'nominees': [{'firstName': 'Chad', 'lastName'...   
3929  {'nominees': [{'firstName': 'Charles', 'lastNa...   
3930  {'nominees': [{'firstName': 'Charles', 'lastNa...   
2574  {'nominees': [{'firstName': 'Charles', 'lastNa...   
990   {'nominees': [{'firstName': 'Clifton', 'lastNa...   
4667  {'nominees': [{'firstName': 'Craig', 'lastName...   
677   {'nominees': [{'firstName': 'Craig', 'lastName...   
1682  {'nominees': [{'firstName': 'Cynthia', 'lastNa...   
1522  {'nominees': [{'firstName': 'Dale', 'lastName'...   

                                                request retrieval_date  
119   {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
778   {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
336   {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
492   {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
1562  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
111   {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
1022  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
1106  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
1108  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
4914  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
78    {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
1034  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
3929  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
3930  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
2574  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
990   {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
4667  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
677   {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
1682  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13  
1522  {'url': 'https://api.congress.gov/v3/nominatio...     2025-07-13


Top duplicate records:


nominee
{'nominees': [{'firstName': 'Mary', 'lastName': 'Abrecht', 'middleName': 'Ellen', 'ordinal': 1, 'state': 'DC'}], 'pagination': {'count': 1}, 'request': {'congress': '101', 'contentType': 'application/json', 'format': 'json', 'number': '1181-0'}}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   7
{'nominees': [{'firstName': 'Robert Samuel', 'lastName': 'Tignor', 'ordinal': 7, 'state': 'DC'}, {'firstName': 'Emmet G.', 'lastName': 'Sullivan', 'ordinal': 6, 'state': 'DC'}, {'firstName': 'Robert Isaac', 'lastName': 'Richter', 'ordinal': 5, 's

In [ ]:
print(f"shape before checking for & dropping duplicated congressional nominee records is { dfs["cong_nominees"].shape}")

# First sort by retrieval_date in ascending order
# Then drop duplicates keeping the last occurrence (which will be the most recent date)
dfs["cong_nominees"] = (dfs["cong_nominees"]
                          .sort_values("retrieval_date")
                          .drop_duplicates(subset=["nominee", "request"], keep='last'))

print(f"shape after checking for & dropping duplicated congressional nominee records is { dfs["cong_nominees"].shape}")

shape before checking for & dropping duplicated congressional nominee records is (5672, 3)
shape after checking for & dropping duplicated congressional nominee records is (5517, 3)


# Flatten JSON-containing congress DataFrames into tabular form

In [ ]:

dfs["cong_nominations"] = flatten_json_dataframe(
    df=dfs["cong_nominations"],
    json_col="nomination",  # column containing the JSON data
    max_list_index=10,      # maximum number of list items to extract
    separator="_"           # separator for nested keys
)

dfs["cong_nominees"]= flatten_json_dataframe(
    df=dfs["cong_nominees"],
    json_col="nominee",
    max_list_index=5
)

2025-07-15 19:32:52.948 | INFO     | nomination_predictor.features:flatten_json_dataframe:316 - Flattening JSON data from column 'nomination' in 5557 rows
2025-07-15 19:33:29.402 | INFO     | nomination_predictor.features:flatten_json_dataframe:342 - Flattening complete. Original columns: 4, New columns: 37
2025-07-15 19:33:29.404 | INFO     | nomination_predictor.features:flatten_json_dataframe:316 - Flattening JSON data from column 'nominee' in 5517 rows
2025-07-15 19:32:57.510 | INFO     | nomination_predictor.features:flatten_json_dataframe:342 - Flattening complete. Original columns: 3, New columns: 34


In [ ]:
# Print summary of available dataframes
print("Available dataframes:")
for name, df in dfs.items():
    print(f"- {name}: {len(df)} rows × {len(df.columns)} columns")

Available dataframes:
- fjc_judges: 4022 rows × 201 columns
- fjc_federal_judicial_service: 4720 rows × 30 columns
- fjc_demographics: 4022 rows × 18 columns
- fjc_education: 8040 rows × 6 columns
- fjc_other_federal_judicial_service: 611 rows × 31 columns
- fjc_other_nominations_recess: 828 rows × 4 columns
- fjc_professional_career: 19003 rows × 4 columns
- cong_nominations: 5557 rows × 37 columns
- cong_nominees: 5517 rows × 34 columns


## Quick peek at all loaded dataframes

In [ ]:
logger.info("Checking for general shape and first handfuls of rows")
for name, df in dfs.items():
    print(f"{name:<35} → {df.shape}")
    print(df.head())  
    print(df.info())

2025-07-15 19:32:57.537 | INFO     | __main__:<module>:1 - Checking for general shape and first handfuls of rows
fjc_judges                          → (4022, 201)
        nid       jid last_name first_name middle_name suffix  birth_month  \
0  13761857  13761857   Abelson       Adam         Ben    NaN          NaN   
1   1393931      3419    Abrams     Ronnie                             NaN   
2   1376976         1   Abruzzo    Matthew          T.                 4.0   
3  13651551  13651551     Abudu      Nancy       Gbana    NaN          NaN   
4   1376981         2   Acheson     Marcus      Wilson                 6.0   

   birth_day birth_year  birth_city  ... degree_(3)  degree_year_(3)  \
0        NaN       1982   Cleveland  ...        NaN              NaN   
1        NaN       1968    New York  ...        NaN              NaN   
2       30.0       1889    Brooklyn  ...        NaN              NaN   
3        NaN       1974  Alexandria  ...        NaN              NaN   
4       

In [ ]:
logger.info("Checking for null values")
    
for name, df in dfs.items():
    print(df.isnull().sum())

2025-07-15 19:32:57.678 | INFO     | __main__:<module>:1 - Checking for null values
nid                                         0
jid                                         0
last_name                                   0
first_name                                  0
middle_name                                35
                                         ... 
school_(5)                               4017
degree_(5)                               4018
degree_year_(5)                          4017
professional_career                         4
other_nominations/recess_appointments    3307
Length: 201, dtype: int64
nid                                     0
sequence                                0
judge_name                              0
court_type                              0
court_name                              0
appointment_title                       0
appointing_president                    0
party_of_appointing_president          39
reappointing_president               4710
party_

# Data cleaning

## Drop duplicated rows (if any) from FJC data

In [ ]:
for name, df in dfs.items():
    if name.startswith("fjc_") and name not in ("fjc_judges", "fjc_demographics", "fjc_other_nominations_recess"):
        dfs[name] = dfs[name].drop_duplicates(subset=["nid", "sequence"])
        print(f"shape of {name} after checking for & dropping duplicated nid sequenced items is { dfs[name].shape}")

shape of fjc_federal_judicial_service after checking for & dropping duplicated nid sequenced items is (4720, 30)
shape of fjc_education after checking for & dropping duplicated nid sequenced items is (8040, 6)
shape of fjc_other_federal_judicial_service after checking for & dropping duplicated nid sequenced items is (611, 31)
shape of fjc_professional_career after checking for & dropping duplicated nid sequenced items is (19003, 4)


## Drop rows whose congressional citations end in -0
All of these I've seen either:

- lack strictly-necessary information such as nomination & confirmation dates,
- lack helpful information such as the person's name, our most straightforward means of linking to FJC data or 
- whatever little information they do have indicates it's not for a position as a judge (e.g. for secretary of defense, assistant secretary to something-or-other-, etc.) 

In [ ]:
from nomination_predictor.features import filter_dash_zero_citations

dfs["cong_nominations"] = filter_dash_zero_citations(dfs["cong_nominations"])

2025-07-15 19:32:57.797 | INFO     | nomination_predictor.features:filter_dash_zero_citations:219 - Found 70 citations ending with '-0'
2025-07-15 19:32:57.803 | INFO     | nomination_predictor.features:filter_dash_zero_citations:225 - Removed 70/5557 records with '-0' citations


## Drop multi-nominee columns

All of these seem associated with non-judge roles (e.g. representatives to the U.N.) and would make our main working dataframe too sparse

In [ ]:
multi_nominee_cols_to_drop = dfs["cong_nominations"].filter(regex=r"nominees_[1-9]").columns
dfs["cong_nominations"] = dfs["cong_nominations"].drop(columns=multi_nominee_cols_to_drop)
print("Remaining nomination columns: ", dfs["cong_nominations"].columns)

multi_nominee_cols_to_drop = dfs["cong_nominees"].filter(regex=r"nominees_[1-9]").columns
dfs["cong_nominees"] = dfs["cong_nominees"].drop(columns=multi_nominee_cols_to_drop)
print("Remaining nominee columns: ", dfs["cong_nominees"].columns)

Remaining nomination columns:  Index(['request', 'retrieval_date', 'is_full_detail', 'actions_count',
       'actions_url', 'authorityDate', 'citation', 'committees_count',
       'committees_url', 'congress', 'description', 'hearings_count',
       'hearings_url', 'latestAction_actionDate', 'latestAction_text',
       'nominationType_isCivilian', 'nominees_0_nomineeCount',
       'nominees_0_ordinal', 'nominees_0_organization',
       'nominees_0_positionTitle', 'nominees_0_url', 'number', 'partNumber',
       'receivedDate', 'updateDate', 'executiveCalendarNumber', 'isList',
       'nominees_0_division', 'nominees_0_introText', 'isPrivileged'],
      dtype='object')
Remaining nominee columns:  Index(['request', 'retrieval_date', 'nominees_0_firstName',
       'nominees_0_lastName', 'nominees_0_middleName', 'nominees_0_ordinal',
       'nominees_0_state', 'pagination_count', 'request_congress',
       'request_contentType', 'request_format', 'request_number',
       'nominees_0_predec

## Dropping unhelpfully uninformative columns

I define these as columns that are fully populated with the same value and wouldn't help modeling.  This still keeps columns if most of it is missing but the one value which is present is the same wherever it's present.

In [ ]:
from nomination_predictor.features import \
    drop_unhelpfully_uninformative_columns

for name, df in dfs.items():
    df = drop_unhelpfully_uninformative_columns(df)
    dfs[name] = df

Columns with limited unique values:
  - 2nd_service_as_chief_judge,_begin_(2): 100% missing values - DROPPING
  - 2nd_service_as_chief_judge,_begin_(3): 100% missing values - DROPPING
  - 2nd_service_as_chief_judge,_begin_(4): 100% missing values - DROPPING
  - 2nd_service_as_chief_judge,_begin_(5): 100% missing values - DROPPING
  - 2nd_service_as_chief_judge,_begin_(6): 100% missing values - DROPPING
  - 2nd_service_as_chief_judge,_end_(2): 100% missing values - DROPPING
  - 2nd_service_as_chief_judge,_end_(3): 100% missing values - DROPPING
  - 2nd_service_as_chief_judge,_end_(4): 100% missing values - DROPPING
  - 2nd_service_as_chief_judge,_end_(5): 100% missing values - DROPPING
  - 2nd_service_as_chief_judge,_end_(6): 100% missing values - DROPPING
  - aba_rating_(4): 1 unique non-null value 'Well Qualified' (0.0% of rows) - KEEPING
  - aba_rating_(5): 100% missing values - DROPPING
  - aba_rating_(6): 100% missing values - DROPPING
  - appointing_president_(5): 1 unique non-nul

## Normalize column names for DataFrames

In [ ]:
print("=== Column Names Before ===")

for name, df in dfs.items():
    print(f"{name:<35} → {df.columns.tolist()}")

=== Column Names Before ===
fjc_judges                          → ['nid', 'jid', 'last_name', 'first_name', 'middle_name', 'suffix', 'birth_month', 'birth_day', 'birth_year', 'birth_city', 'birth_state', 'death_month', 'death_day', 'death_year', 'death_city', 'death_state', 'gender', 'race_or_ethnicity', 'court_type_(1)', 'court_name_(1)', 'appointment_title_(1)', 'appointing_president_(1)', 'party_of_appointing_president_(1)', 'reappointing_president_(1)', 'party_of_reappointing_president_(1)', 'aba_rating_(1)', 'seat_id_(1)', 'statute_authorizing_new_seat_(1)', 'recess_appointment_date_(1)', 'nomination_date_(1)', 'committee_referral_date_(1)', 'hearing_date_(1)', 'judiciary_committee_action_(1)', 'committee_action_date_(1)', 'senate_vote_type_(1)', 'ayes/nays_(1)', 'confirmation_date_(1)', 'commission_date_(1)', 'service_as_chief_judge,_begin_(1)', 'service_as_chief_judge,_end_(1)', '2nd_service_as_chief_judge,_begin_(1)', '2nd_service_as_chief_judge,_end_(1)', 'senior_status_date_(

In [ ]:
# call features.py's normalize_columns function on all DataFrames in dfs, and strip leading and trailing whitespace in all strings
from nomination_predictor.features import normalize_dataframe_columns

for name, df in dfs.items():
    df = normalize_dataframe_columns(df)
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    dfs[name] = df

In [ ]:
print("=== Column Names After ===")

for name, df in dfs.items():
    print(f"{name:<35} → {df.columns.tolist()}")

=== Column Names After ===
fjc_judges                          → ['nid', 'jid', 'last_name', 'first_name', 'middle_name', 'suffix', 'birth_month', 'birth_day', 'birth_year', 'birth_city', 'birth_state', 'death_month', 'death_day', 'death_year', 'death_city', 'death_state', 'gender', 'race_or_ethnicity', 'court_type_(1)', 'court_name_(1)', 'appointment_title_(1)', 'appointing_president_(1)', 'party_of_appointing_president_(1)', 'reappointing_president_(1)', 'party_of_reappointing_president_(1)', 'aba_rating_(1)', 'seat_id_(1)', 'statute_authorizing_new_seat_(1)', 'recess_appointment_date_(1)', 'nomination_date_(1)', 'committee_referral_date_(1)', 'hearing_date_(1)', 'judiciary_committee_action_(1)', 'committee_action_date_(1)', 'senate_vote_type_(1)', 'ayes/nays_(1)', 'confirmation_date_(1)', 'commission_date_(1)', 'service_as_chief_judge,_begin_(1)', 'service_as_chief_judge,_end_(1)', '2nd_service_as_chief_judge,_begin_(1)', '2nd_service_as_chief_judge,_end_(1)', 'senior_status_date_(1

## Left-merge nominees table onto nominations table
This will make the cong_noms dataframe we'll use for most of our congress data operations below in this notebook.

In [ ]:
from nomination_predictor.features import merge_nominees_onto_nominations

dfs["cong_noms"] = pd.DataFrame() # initialize at this scope so the one created in the below "try" block persists for further-down notebook cells


try:
    # Assuming cong_nominations and cong_nominees dataframes are already loaded
    dfs["cong_noms"] = merge_nominees_onto_nominations(dfs["cong_nominations"], dfs["cong_nominees"])
    
    
    # Report on the merge results
    logger.info(f"Original nominations shape: {dfs['cong_nominations'].shape}")
    logger.info(f"Original nominees shape: {dfs['cong_nominees'].shape}")
    logger.info(f"Merged dataframe shape: {dfs['cong_noms'].shape}")
    
except Exception as e:
    logger.error(f"Error in merge process: {e}")
    
# Show sample of the merged dataframe
display(dfs["cong_noms"].head())

2025-07-15 19:32:58.729 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:529 - Extracted 5517 URLs from nominees request column (100.0% of rows)
2025-07-15 19:32:58.731 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:553 - Nominations dataframe has 5487 non-null URLs (100.0% of rows)
2025-07-15 19:32:58.743 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:569 - Merged dataframe has 5487 rows
2025-07-15 19:32:58.743 | INFO     | nomination_predictor.features:merge_nominees_onto_nominations:570 - Successfully matched 5487 nominations with nominees (100.0%)
2025-07-15 19:32:58.744 | INFO     | __main__:<module>:12 - Original nominations shape: (5487, 24)
2025-07-15 19:32:58.745 | INFO     | __main__:<module>:13 - Original nominees shape: (5517, 12)
2025-07-15 19:32:58.745 | INFO     | __main__:<module>:14 - Merged dataframe shape: (5487, 35)


request  actions_count  \
0  {'congress': '118', 'contentType': 'applicatio...              6   
1  {'congress': '104', 'contentType': 'applicatio...              2   
2  {'congress': '104', 'contentType': 'applicatio...              2   
3  {'congress': '104', 'contentType': 'applicatio...              5   
4  {'congress': '104', 'contentType': 'applicatio...              5   

                                         actions_url authoritydate citation  \
0  https://api.congress.gov/v3/nomination/118/201...    2025-05-12   PN2013   
1  https://api.congress.gov/v3/nomination/104/556...          None    PN556   
2  https://api.congress.gov/v3/nomination/104/577...          None    PN577   
3  https://api.congress.gov/v3/nomination/104/581...          None    PN581   
4  https://api.congress.gov/v3/nomination/104/582...          None    PN582   

   committees_count                                     committees_url  \
0               1.0  https://api.congress.gov/v3/nomination/118/201...   
1               1.0  https://api.congress.gov/v3/nomination/104/556...   
2               1.0  https://api.congress.gov/v3/nomination/104/577...   
3               1.0  https://api.congress.gov/v3/nomination/104/581...   
4               1.0  https://api.congress.gov/v3/nomination/104/582...   

   congress                                        description  \
0       118  Nicholas George Miranda, of the District of Co...   
1       104  Bruce W. Greer, of Florida, to be United State...   
2       104  Gerald M. Shea, of the District of Columbia, t...   
3       104  Joseph Francis Baca, of New Mexico, to be a Me...   
4       104  Bruce D. Black, of New Mexico, to be United St...   

   hearings_count  ... nominees_0_firstname nominees_0_lastname  \
0             1.0  ...             Nicholas             Miranda   
1             NaN  ...                Bruce               Greer   
2             NaN  ...               Gerald                Shea   
3             NaN  ...               Joseph                Baca   
4             NaN  ...                Bruce               Black   

  nominees_0_middlename  nominees_0_ordinal nominees_0_state pagination_count  \
0                George                   1               DC                1   
1                    W.                   1               FL                1   
2                    M.                   1               DC                1   
3               Francis                   1               NM                1   
4                    D.                   1               NM                1   

  request_congress  request_number nominees_0_predecessorname  \
0              118            2013                       None   
1              104             556             James W. Kehoe   
2              104             577                       None   
3              104             581                       None   
4              104             582     Juan Guerrero Burciaga   

  nominees_0_suffix  
0              None  
1              None  
2              None  
3              None  
4              None  

[5 rows x 35 columns]

## Drop non-judge nominations based on organization or position title

I was curious whether the `nominee_0_organization` field expressed where the nominee was coming from, vs. where they were being nominated to.

Because if it was where they're being nominated from, then deleting rows with that outside the judiciary, tax courts, etc. would be erasure of people getting their first judgeship nomination.

Whereas if it was where they're being nominated to, then deleting rows with that outside the judiciary, tax courts, etc. would be a quick, simple way to clear out non-judge roles.

To check, for this...

In [ ]:
# display rows whose "description" field contains the word "judge", but whose "nominees_0_organization" field does not contain either of the words "Judiciary" or "Court"
to_display = dfs["cong_noms"][dfs["cong_noms"]["description"].str.contains("judge", case=False, na=False) & ~dfs["cong_noms"]["nominees_0_organization"].str.contains("Judiciary", case=False, na=False) & ~dfs["cong_noms"]["nominees_0_organization"].str.contains("Court", case=False, na=False)]
to_display.head()

request  actions_count  \
1127  {'congress': '106', 'contentType': 'applicatio...              6   
2845  {'congress': '115', 'contentType': 'applicatio...              5   
2996  {'congress': '116', 'contentType': 'applicatio...              6   
5423  {'congress': '112', 'contentType': 'applicatio...              5   
5424  {'congress': '112', 'contentType': 'applicatio...              5   

                                            actions_url authoritydate  \
1127  https://api.congress.gov/v3/nomination/106/127...          None   
2845  https://api.congress.gov/v3/nomination/115/246...          None   
2996  https://api.congress.gov/v3/nomination/116/209...          None   
5423  https://api.congress.gov/v3/nomination/112/111...          None   
5424  https://api.congress.gov/v3/nomination/112/112...          None   

     citation  committees_count  \
1127    PN127               1.0   
2845   PN2464               1.0   
2996    PN209               1.0   
5423   PN1119               1.0   
5424   PN1120               1.0   

                                         committees_url  congress  \
1127  https://api.congress.gov/v3/nomination/106/127...       106   
2845  https://api.congress.gov/v3/nomination/115/246...       115   
2996  https://api.congress.gov/v3/nomination/116/209...       116   
5423  https://api.congress.gov/v3/nomination/112/111...       112   
5424  https://api.congress.gov/v3/nomination/112/112...       112   

                                            description  hearings_count  ...  \
1127  Robert A. Katzmann, of New York, to be United ...             NaN  ...   
2845  Lisa M. Schenck, of Virginia, to be a Judge of...             NaN  ...   
2996  Lisa M. Schenck, of Virginia, to be a Judge of...             NaN  ...   
5423  William B. Pollard, III, of New York, to be a ...             NaN  ...   
5424  Scott L. Silliman, of North Carolina, to be a ...             NaN  ...   

     nominees_0_firstname nominees_0_lastname nominees_0_middlename  \
1127               Robert            Katzmann                    A.   
2845                 Lisa             Schenck                    M.   
2996                 Lisa             Schenck                    M.   
5423              William             Pollard                    B.   
5424                Scott            Silliman                    L.   

      nominees_0_ordinal nominees_0_state pagination_count request_congress  \
1127                   1               NY                1              106   
2845                   1               VA                1              115   
2996                   1               VA                1              116   
5423                   1               NY                1              112   
5424                   1               NC                1              112   

      request_number nominees_0_predecessorname nominees_0_suffix  
1127             127              Jon O. Newman              None  
2845            2464                       None              None  
2996             209                       None              None  
5423            1119                       None               III  
5424            1120                       None              None  

[5 rows x 35 columns]

...so that told me the organization field is where they're coming from, not where they're being nominated to.  And includes a few people trying to move from places like Department of Defense (or who just plain had the organization field missing from our data) to get into a position as a judge.  So the Organization field wasn't an accurate selector for getting rid of non-judge rows.

Instead will try to use the nominee_0_positiontitle field and the description text to filter out non-judicial nominations:

In [ ]:
# Filter out non-judicial nominations using the function from features.py
from nomination_predictor.features import filter_non_judicial_nominations

dfs["cong_noms"] = filter_non_judicial_nominations(dfs["cong_noms"])

2025-07-15 19:32:58.847 | INFO     | nomination_predictor.features:filter_non_judicial_nominations:185 - Found 1103 unique citations with non-judicial titles
2025-07-15 19:32:58.851 | INFO     | nomination_predictor.features:filter_non_judicial_nominations:191 - Removed 3476/5487 corresponding records


## Populate a few new, cleaner columns from straightforwardly-parsable data

In [ ]:
from nomination_predictor.name_matching import fill_vacancy_reason_column

dfs["cong_noms"] = fill_vacancy_reason_column(dfs["cong_noms"])

2025-07-15 19:32:58.883 | INFO     | nomination_predictor.name_matching:fill_vacancy_reason_column:404 - Extracted 2011 vacancy reasons from descriptions


In [ ]:
# fill missing values from predecessor column
from nomination_predictor.name_matching import fill_predecessor_column

dfs["cong_noms"] = fill_predecessor_column(dfs["cong_noms"])

In [ ]:
# create a new column in dfs["fjc_demographics"] titled "fjc_biography_url"
# whose contents are simply the string "http://www.fjc.gov/node/" concatenated before whatever integer # can be read from 
# that same row's "nid" column (or empty string if nid is not an integer)

svc = dfs["fjc_demographics"]

svc["fjc_biography_url"] = (
    "http://www.fjc.gov/node/" +
    pd.to_numeric(svc["nid"], errors="coerce")     # turn non-ints into NaN
      .dropna()                                    # keep only numeric nids
      .astype(int)                                 # cast to int for clean string
      .astype(str)   
)
dfs["fjc_demographics"] = svc

## Convert date strings to datetime objects

In [ ]:
# for any columns which contain certain keywords in their column name and contain string values, convert from string to datetime
datetime_related_keywords = ("date", "year", "month")

for name, df in dfs.items():
    for col in df.columns:
        if any(keyword in col for keyword in datetime_related_keywords) and df[col].dtype == "object":
            logger.info(f"Converting {col} to datetime for {name}")
            df[col] = pd.to_datetime(df[col], errors="coerce")

2025-07-15 19:32:58.940 | INFO     | __main__:<module>:7 - Converting birth_year to datetime for fjc_judges
2025-07-15 19:32:58.948 | INFO     | __main__:<module>:7 - Converting recess_appointment_date_(1) to datetime for fjc_judges
2025-07-15 19:32:58.953 | INFO     | __main__:<module>:7 - Converting nomination_date_(1) to datetime for fjc_judges
2025-07-15 19:32:58.957 | INFO     | __main__:<module>:7 - Converting committee_referral_date_(1) to datetime for fjc_judges
2025-07-15 19:32:58.959 | INFO     | __main__:<module>:7 - Converting hearing_date_(1) to datetime for fjc_judges
2025-07-15 19:32:58.962 | INFO     | __main__:<module>:7 - Converting committee_action_date_(1) to datetime for fjc_judges
2025-07-15 19:32:58.964 | INFO     | __main__:<module>:7 - Converting confirmation_date_(1) to datetime for fjc_judges
2025-07-15 19:32:58.966 | INFO     | __main__:<module>:7 - Converting commission_date_(1) to datetime for fjc_judges
2025-07-15 19:32:58.969 | INFO     | __main__:<modul

## Normalize several columns' string values to make matching them later easier

In [ ]:
from nomination_predictor.name_matching import normalize_text

keywords_which_denote_string_columns_to_normalize = ("court", "circuit", "district", "description", "name")

for name, df in dfs.items():
    for col in df.columns:
        if any(keyword in col.casefold() for keyword in keywords_which_denote_string_columns_to_normalize) and df[col].dtype == object:
            logger.info(F"Normalizing all values within column named {col} in {name}")
            df[col] = df[col].apply(normalize_text)

2025-07-15 19:32:59.186 | INFO     | __main__:<module>:8 - Normalizing all values within column named last_name in fjc_judges
2025-07-15 19:32:59.196 | INFO     | __main__:<module>:8 - Normalizing all values within column named first_name in fjc_judges
2025-07-15 19:32:59.204 | INFO     | __main__:<module>:8 - Normalizing all values within column named middle_name in fjc_judges
2025-07-15 19:32:59.209 | INFO     | __main__:<module>:8 - Normalizing all values within column named court_type_(1) in fjc_judges
2025-07-15 19:32:59.216 | INFO     | __main__:<module>:8 - Normalizing all values within column named court_name_(1) in fjc_judges
2025-07-15 19:32:59.225 | INFO     | __main__:<module>:8 - Normalizing all values within column named court_type_(2) in fjc_judges
2025-07-15 19:32:59.231 | INFO     | __main__:<module>:8 - Normalizing all values within column named court_name_(2) in fjc_judges
2025-07-15 19:32:59.234 | INFO     | __main__:<module>:8 - Normalizing all values within column

## Count and display columns for which no value is unique:
looks like our left-merge of the two congress dataframes causes some blank columns, and we still have some that are populated but unhelpful..

In [ ]:
# display counts of unique values in DataFrame columns:
for name, df in dfs.items():
    for col in sorted(df.columns):
        if df[col].nunique() <1:
            print(f"{name} - {col}: {df[col].nunique()} unique values")

cong_noms - islist: 0 unique values
cong_noms - nominees_0_introtext: 0 unique values


...so now's an okay time to delete them

In [ ]:
for name, df in dfs.items():
    df = drop_unhelpfully_uninformative_columns(df)
    dfs[name] = df

Columns with limited unique values:
  - aba_rating_(4): 1 unique non-null value 'Well Qualified' (0.0% of rows) - KEEPING
  - appointing_president_(5): 1 unique non-null value 'Harry S Truman' (0.0% of rows) - KEEPING
  - appointing_president_(6): 1 unique non-null value 'Harry S Truman' (0.0% of rows) - KEEPING
  - appointment_title_(4): 1 unique non-null value 'Judge' (0.3% of rows) - KEEPING
  - appointment_title_(5): 1 unique non-null value 'Judge' (0.0% of rows) - KEEPING
  - appointment_title_(6): 1 unique non-null value 'Judge' (0.0% of rows) - KEEPING
  - ayes/nays_(4): 1 unique non-null value '' (0.3% of rows) - KEEPING
  - ayes/nays_(5): 1 unique non-null value '' (0.0% of rows) - KEEPING
  - ayes/nays_(6): 1 unique non-null value '' (0.0% of rows) - KEEPING
  - commission_date_(5): 1 unique non-null value '1949-02-02 00:00:00' (0.0% of rows) - KEEPING
  - commission_date_(6): 1 unique non-null value '1949-02-02 00:00:00' (0.0% of rows) - KEEPING
  - committee_action_date_(5)

# Name-matching FJC judges to Congress.gov nominees

## For confirmed judges

### Supplementing with additional columns to aid matching

In [ ]:
# add a "full_name_from_description" and a "location_of_origin_from_description" columns to the dfs["cong_noms"] dataframe which regex-captures the first segments of the same dfs["cong_noms"] dataframe row's "description" string, 
# i.e. captures name before the first appearances of the phrases ", of " or ", of the "
# and captures location from the second segment of the same dfs["cong_noms"] dataframe row's "description" string
# i.e. captures between the above-seen phrase ", of " or ", of the " through to the phrase ", to be "
# examples: 
# melissa damian, of florida, to be ...  gets captured into those new columns as "melissa damian" and "florida"
# nicole g. bernerr of maryland, to be united... gets captured into those new columns as "nicole g. bernerr" and "maryland"
# kirk edward sherriff, of california, to be united... gets captured into those new columns as "kirk edward sherriff" and "california"
# sherri malloy beatty-arthur, of the district of columbia, for... gets captured into those new columns as "sherri malloy beatty-arthur" and "district of columbia"

# Extract full_name_from_description and location_of_origin_from_description from description field
from nomination_predictor.features import extract_name_and_location_columns

# Apply the extraction function to cong_noms dataframe
if 'cong_noms' in dfs:
    dfs['cong_noms'] = extract_name_and_location_columns(dfs['cong_noms'])
    
    # Display sample results to verify extraction
    sample_cols = ['description', 'full_name_from_description', 'location_of_origin_from_description']
    display(dfs['cong_noms'][sample_cols].head(10))
    
    # Report extraction statistics
    total_rows = len(dfs['cong_noms'])
    name_filled = dfs['cong_noms']['full_name_from_description'].notna().sum()
    location_filled = dfs['cong_noms']['location_of_origin_from_description'].notna().sum()
    
    logger.info(f"Extracted names for {name_filled}/{total_rows} records ({name_filled/total_rows:.1%})")
    logger.info(f"Extracted locations for {location_filled}/{total_rows} records ({location_filled/total_rows:.1%})")
else:
    logger.error("Error: 'cong_noms' dataframe not found in dfs dictionary.")

2025-07-15 19:32:59.861 | INFO     | nomination_predictor.features:extract_name_and_location_columns:693 - Extracted 2011/2011 (100.0%) names and 2011/2011 (100.0%) locations


description  \
0   nicholas george miranda, of the district of co...   
1   bruce w. greer, of florida, to be united state...   
4   bruce d. black, of new mexico, to be united st...   
8   susan j. dlott, of ohio, to be united states d...   
9   james allan hurd, jr., of the virgin islands, ...   
12  hilda g. tagle, of texas, to be united states ...   
15  merrick b. garland, of maryland, to be united ...   
18  hugh lawson, of georgia, to be united states d...   
20  michael r. murphy, of utah, to be united state...   
21  john h. bingler, jr., of pennsylvania, to be u...   

   full_name_from_description location_of_origin_from_description  
0     nicholas george miranda                district of columbia  
1              bruce w. greer                             florida  
4              bruce d. black                          new mexico  
8              susan j. dlott                                ohio  
9       james allan hurd, jr.                      virgin islands  
12             hilda g. tagle                                 xas  
15         merrick b. garland                            maryland  
18                hugh lawson                             georgia  
20          michael r. murphy                                 uta  
21       john h. bingler, jr.                        pennsylvania

2025-07-15 19:32:59.872 | INFO     | __main__:<module>:27 - Extracted names for 2011/2011 records (100.0%)
2025-07-15 19:32:59.873 | INFO     | __main__:<module>:28 - Extracted locations for 2011/2011 records (100.0%)


### Performing the matching operations

In [ ]:
from nomination_predictor.name_matching import perform_exact_name_matching

results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices= perform_exact_name_matching(
    congress_df=dfs["cong_noms"],
    fjc_df=dfs["fjc_federal_judicial_service"],
    congress_name_col="full_name_from_description",
    fjc_name_col="judge_name"
)

2025-07-15 19:32:59.888 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:123 - Starting exact name matching with 2011 Congress records and 4720 FJC records
2025-07-15 19:33:00.206 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:142 - Performing first-pass join on last and first name
2025-07-15 19:33:00.227 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:149 - Found 1246 total records with last+first name matches
2025-07-15 19:33:00.235 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:177 - Found 27 ambiguous matches, attempting middle initial disambiguation
2025-07-15 19:33:00.236 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:181 - Samples of pending ambiguous rows
2025-07-15 19:33:00.236 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:182 -                                                 request  actions_count  \
670   

In [ ]:
# Show results
results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices.head()

congress_index       congress_name             fjc_name        nid  \
0               4      bruce d. black      black, bruce d.  1377866.0   
1               8      susan j. dlott      dlott, susan j.  1380076.0   
2              12      hilda g. tagle      tagle, hilda g.  1390611.0   
3              15  merrick b. garland  garland, merrick b.  1381046.0   
4              18         hugh lawson         lawson, hugh  1383696.0   

            match_type  ambiguous  
0  first_and_last_name      False  
1  first_and_last_name      False  
2  first_and_last_name      False  
3  first_and_last_name      False  
4  first_and_last_name      False

In [ ]:
# keep only *unambiguous* pairs
nid_map = (
    results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices[~results_of_name_matching_to_bridge_nids_to_congress_dataframe_indices["ambiguous"]]        # drop rows still ambiguous
      .set_index("congress_index")["nid"]
)

In [ ]:
# at long last, we have a way to bridge the gap between the congress.gov data and the fjc data

# we can now use the nid_map to add the nid column to the congress.gov data
dfs["cong_noms"]["nid"] = dfs["cong_noms"].index.to_series().map(nid_map)
cong_noms = dfs["cong_noms"]

## For unconfirmed judges

In practice, given the dataframes as I've got them as of typing this, this section doesn't find any remaining unconfirmed judges to match.

What this section _did_ accomplish was showing me that the presence of diacritical marks such as "ñ" or "é" in names was misleading the matching process.

Discovering and addressing that in much-earlier data-normalizing cells led to getting more matches in our confirmed-judges-matching notebook section.

### Supplementing with additional columns to aid matching

In [ ]:
from nomination_predictor.name_matching import prep_fjc_other

dfs["fjc_other_nominations_recess"] = prep_fjc_other(fjc_other_df=dfs["fjc_other_nominations_recess"])

In [ ]:
fjc_other_supplemented =dfs["fjc_other_nominations_recess"]

### Inspecting for the unconfirmed nominee matching possibilities

In [ ]:
from nomination_predictor.features import link_unconfirmed_nominations

dfs["cong_noms"] = link_unconfirmed_nominations(dfs["cong_noms"], dfs["fjc_other_nominations_recess"])

2025-07-15 19:33:00.604 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:123 - Starting exact name matching with 304 Congress records and 828 FJC records
2025-07-15 19:33:00.671 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:142 - Performing first-pass join on last and first name
2025-07-15 19:33:00.682 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:149 - Found 0 total records with last+first name matches
2025-07-15 19:33:00.682 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:153 - NO last+first name matches found. Checking last-name-only matches for diagnosis...
2025-07-15 19:33:00.687 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:157 - Found 92 last-name-only matches
2025-07-15 19:33:00.688 | INFO     | nomination_predictor.name_matching:perform_exact_name_matching:159 - Showing up to first 10 last-name-only matches:
2025-07-15 19:33:00.689 | 

### Deciding not to merge the "Other Nominations" dataframe (at least not yet)
In theory this could get us the fjc's perspective on more nominees who didn't get confirmed.  

In practice my runs of name-matching didn't find any unambiguous matches from this dataframe to the congress one.  

So I put the idea of merging it, too, on hold.  Can try another day, maybe after seeing whether the additional data would help, or if ever discovering something major could be fixed/improved about the name matcher.

# Combining the rest of the FJC data now that our congress dataframe has been enriched with FJC nid

## Handling nominees' education and job history

Before we combine FJC data, we have to consider whether/how to handle judges' education, job history, age, ABA rating, etc.  Most/all of the data in the "demographics" dataframe is unchanging over time, but that's very much _not_ true of the other dataframes.

The simplest way to handle it would be to left-merge on "nid" and only take the most recently-dated row, or row with the highest sequence number.  In most cases this would likely land on keeping the most prestigious degree or job.

However, it is entirely likely a judge's education or job history has changed substantially since their first nomination, and affected their qualifications for each later nomination.

All of these indicate to me that it's worth merging onto each row that judge's position, education, etc., not as of the most recent records available, but instead _as of when they were nominated._

That means we can't do a too-simple left-join of all of our FJC data.  Instead, now that we've done the step of matching NIDs to congress' data on nominations, we can use the "received date" for each congress citation as a cutoff date for when we lookup education and job records by "nid" -- so we can avoid mistakenly linking to a citation any employment & job records dated after that cutoff date.

Thankfully we do have the school, degree, and degree_year in the education record, for both their bachelors and their masters and their associate degree(s) and LLB and J.D. etc., so we can look that up.  The education dataframe even comes with a "sequence" number for each education record, which is another indicator of chronological order in addition to degree_year for any given "nid" lookup for a judge.

Job history is more challenging to deal with because literally every row entry in that dataframe lists it uniquely, but we do have the data available.  My earliest attempts to feature-engineer with it include looking for keywords in it, then creating boolean features for whether they did/didn't have experience in common-phrase-identifiable positions such as "Private practice" or "Attorney general" or "Navy" or "Army" etc. Theoretically a parser can look for the year spreads listed there as a rough indicator of amounts of experience gleaned from each professional role & when, but that may be too complicated for me to accomplish by the time I'm first presenting this work.

In [ ]:

import pandas as pd

dfs["cong_noms"] = dfs["cong_noms"].merge(dfs["fjc_demographics"],  on="nid", how="left", suffixes=("", "_fjcdemographics"))

In [ ]:
from nomination_predictor.time_aware_analysis import (
    merge_latest_career, merge_latest_education, merge_nearest_fed_service)

dfs["cong_noms"] = merge_latest_education(dfs["cong_noms"], dfs["fjc_education"])
dfs["cong_noms"] = merge_latest_career(dfs["cong_noms"], dfs["fjc_professional_career"])

In [ ]:
dfs["cong_noms"] = merge_nearest_fed_service(dfs["cong_noms"], dfs["fjc_federal_judicial_service"])

## Compressing educational, professional, and federal service career history columns

This section exists because it would help to get rid of a bunch of columns that are very empty for the vast majority of rows, yet still hold relevance for the rare rows where they're populated.  Columns like the fifth graduate degree one or few judges got, or the Nth job a frequent career-hopper held.

Ideally before we get rid of these largely-empty columns, it would help to gather the durations of federal and/or professional roles in case we want to visualize things like typical role durations for a given seat type, whether roles tend to get longer vs. briefer with age, whether roles last longer in some court types or circuits than others, etc.  Parsing the duration of professional roles from those widely-varying text strings sounded more difficult than is worth it quite yet, so I haven't pushed this idea too far.  For now, it'd be easier & hopefully helpful-enough to just note the number of jobs they've had in a "num_jobs" column, and the number of graduate degrees they've had in a "num_grad_degrees" column.

Durations of federal service roles are easier with the data we have: for each row, find the durations between confirmation date and termination date for each of the times they've had a job in federal service so far, put that duration into a new "days_federal_service_experience" column.  When putting a value in that new column, sum up the multiple diffs on their row if they've held multiple federal service roles.

FIXME: the above way we handled time-aware data may be making the assumption that only roles which _ended_ before nomination are worth considering in their experience.  That's often not true -- a nominee may have been nominated while still working on a job relevant to their qualifications, one they haven't gotten "terminated" from yet because they haven't left it.  May have to go back and check on how the time_aware_merged_df was constructed to check whether this is the case.

Then finally delete the no-longer-needed largely-empty columns we condensed.

# Feature engineering 

In [ ]:
from datetime import date

from nomination_predictor.time_aware_analysis import (
    congress_number, congress_session, days_into_current_term,
    days_until_next_midterm_election, days_until_next_presidential_election,
    fill_missing_appointing_presidents,
    fill_missing_party_of_appointing_presidents, normalize_party_codes,
    presidential_term_index)

df = dfs["cong_noms"].copy()
df["receiveddate"] = pd.to_datetime(df["receiveddate"])   # ensure datetime

### Primary target variable to train a model to predict:
number of days between receiveddate and latestaction_actiondate

In [ ]:
print("Calculating days from nomination to latest action...")

# Initialize the new column
df['days_nom_to_latest_action'] = pd.NA

# Create mask for rows with both dates available
valid_dates_mask = df['receiveddate'].notna() & df['latestaction_actiondate'].notna()

if valid_dates_mask.any():
    # Calculate the difference in days
    df.loc[valid_dates_mask, 'days_nom_to_latest_action'] = (
        (df.loc[valid_dates_mask, 'latestaction_actiondate'] - 
         df.loc[valid_dates_mask, 'receiveddate']).dt.days
    )
    
    # Optional: Handle negative values (if any latest action dates are before received dates)
    negative_days_mask = df['days_nom_to_latest_action'] < 0
    if negative_days_mask.any():
        logger.warning(f"{negative_days_mask.sum()} rows have negative duration (latest action before received date)")
        logger.warning("Sample of problematic rows:")
        display(df.loc[negative_days_mask, ['receiveddate', 'latestaction_actiondate', 'days_nom_to_latest_action']].head(3))
        
        # You can decide to either keep negative values or set them to NA
        # df.loc[negative_days_mask, 'days_nom_to_latest_action'] = pd.NA  # Uncomment to remove negative values

# Print summary statistics
days_count = df['days_nom_to_latest_action'].notna().sum()
logger.info(f"Successfully calculated duration for {days_count} nominees ({days_count/len(df):.1%} of dataset)")

if days_count > 0:
    logger.info(f"Duration statistics (days):")
    logger.info(f"- Min: {df['days_nom_to_latest_action'].min()} days")
    logger.info(f"- Max: {df['days_nom_to_latest_action'].max()} days")
    logger.info(f"- Mean: {df['days_nom_to_latest_action'].mean():.1f} days")
    logger.info(f"- Median: {df['days_nom_to_latest_action'].median():.1f} days")

Calculating days from nomination to latest action...
2025-07-15 19:33:09.001 | INFO     | __main__:<module>:28 - Successfully calculated duration for 2011 nominees (100.0% of dataset)
2025-07-15 19:33:09.002 | INFO     | __main__:<module>:31 - Duration statistics (days):
2025-07-15 19:33:09.004 | INFO     | __main__:<module>:32 - - Min: 0 days
2025-07-15 19:33:09.005 | INFO     | __main__:<module>:33 - - Max: 727 days
2025-07-15 19:33:09.006 | INFO     | __main__:<module>:34 - - Mean: 135.7 days
2025-07-15 19:33:09.008 | INFO     | __main__:<module>:35 - - Median: 103.0 days


## Pull yea/nay votes and confirmed/returned from description into their own columns

In [ ]:
from nomination_predictor.latestaction_parser import enrich_latest_action

dfs["cong_noms"] = enrich_latest_action(dfs["cong_noms"])

In [ ]:
# presidency- and elections-timeline-related
df["pres_term_idx"]  = df["receiveddate"].apply(presidential_term_index)
df["days_into_pres_term"] = df["receiveddate"].apply(days_into_current_term)
df["days_to_next_pres_election"] = df["receiveddate"].apply(days_until_next_presidential_election)
df["days_to_next_midterm_election"]  = df["receiveddate"].apply(days_until_next_midterm_election)
df["congress_num"] = df["receiveddate"].apply(congress_number)
df["congress_session"] = df["receiveddate"].apply(congress_session)

In [ ]:
print(dfs["cong_noms"].head())

                                             request  actions_count  \
0  {'congress': '118', 'contentType': 'applicatio...              6   
1  {'congress': '104', 'contentType': 'applicatio...              2   
2  {'congress': '104', 'contentType': 'applicatio...              5   
3  {'congress': '104', 'contentType': 'applicatio...              6   
4  {'congress': '104', 'contentType': 'applicatio...              2   

                                         actions_url authoritydate citation  \
0  https://api.congress.gov/v3/nomination/118/201...    2025-05-12   PN2013   
1  https://api.congress.gov/v3/nomination/104/556...           NaT    PN556   
2  https://api.congress.gov/v3/nomination/104/582...           NaT    PN582   
3  https://api.congress.gov/v3/nomination/104/587...           NaT    PN587   
4  https://api.congress.gov/v3/nomination/104/551...           NaT    PN551   

   committees_count                                     committees_url  \
0               1.0  htt

In [ ]:
df = fill_missing_appointing_presidents(df) 

df = fill_missing_party_of_appointing_presidents(df)

df = normalize_party_codes(df, party_columns=['party_of_appointing_president', 'senate_party', 'house_party'])

# Show some examples for each seat level for validation
print("\nExample nominations from each appointing president:")
for prez in df["appointing_president"].unique():
    # Sample data - get both description and receiveddate 
    sample_rows = df[df["appointing_president"] == prez].sample(
        min(2, df[df["appointing_president"] == prez].shape[0])
    )
    
    print(f"\n{prez.upper()} examples:")
    for _, row in sample_rows.iterrows():
        # Format the date nicely
        date_str = row["receiveddate"].strftime("%B %d, %Y") if pd.notna(row["receiveddate"]) else "No date"
        
        # Print description with date
        print(f"  • [{date_str}] {row['description']}")

2025-07-15 19:33:09.257 | INFO     | nomination_predictor.time_aware_analysis:fill_missing_appointing_presidents:519 - Filled 1102 missing appointing president values using nomination dates
2025-07-15 19:33:09.269 | INFO     | nomination_predictor.time_aware_analysis:fill_missing_party_of_appointing_presidents:558 - Filled 1102 missing party of appointing president values using nomination dates
2025-07-15 19:33:09.283 | WARNING  | nomination_predictor.time_aware_analysis:normalize_party_codes:677 - Unmapped party value 'Jeffersonian Republican' found in column 'party_of_appointing_president' at rows [265, 621] - kept as-is
2025-07-15 19:33:09.285 | INFO     | nomination_predictor.time_aware_analysis:normalize_party_codes:686 - Normalized 2011 party codes in column 'party_of_appointing_president'
2025-07-15 19:33:09.286 | WARNING  | nomination_predictor.time_aware_analysis:normalize_party_codes:629 - Column 'senate_party' not found in DataFrame, skipping
2025-07-15 19:33:09.286 | WARNIN

## Judges' ages (to ask things like "How much does judge's age affect approval?")

Among the hypotheses is that older judges tend to get approved faster because there's not as much concern they'll live long enough to have as much of a total impact over their time in office.

In [ ]:
# do we even have enough birthdays to get statistical significance or help model training?
print(f"Merged dataframe has {df['birth_day'].notna().mean()*100}% of rows with birth day")
print(f"Merged dataframe has {df['birth_month'].notna().mean()*100}% of rows with birth month")
print(f"Merged dataframe has {df['birth_year'].notna().mean()*100}% of rows with birth year")

Merged dataframe has 9.000497265042268% of rows with birth day
Merged dataframe has 9.000497265042268% of rows with birth month
Merged dataframe has 60.31824962705122% of rows with birth year


That tells me the FJC didn't offer us enough data yet to get granular down to the day.  At best we can approximate by year.  Let's pretend for simplicity that everyone was born exactly in the middle of their birth year.

In [ ]:
# Age at nomination - using only birth year 

# Reference date for "future" check
yesterday = pd.Timestamp.today().normalize() - pd.Timedelta(days=1)

# Initialize column
df['age_at_nom'] = pd.NA

# Process only rows that have birth_year available
birth_year_mask = df['birth_year'].notna()

if birth_year_mask.any():
    # Since birth_year is already a datetime, just use July 1st of that year
    df['birth_date'] = pd.NaT
    
    # Extract just the year from the timestamp and create a July 1st date
    df.loc[birth_year_mask, 'birth_date'] = df.loc[birth_year_mask, 'birth_year'].apply(
        lambda ts: pd.Timestamp(year=ts.year, month=7, day=1)
    )
    
    # Filter out any future dates
    future_mask = df['birth_date'] > yesterday
    if future_mask.any():
        print(f"Warning: {future_mask.sum()} birth dates were in the future and set to NaT")
        df.loc[future_mask, 'birth_date'] = pd.NaT
    
    # Calculate age in years (as decimal)
    valid_mask = df['birth_date'].notna() & df['receiveddate'].notna()
    df.loc[valid_mask, 'age_at_nom'] = (
        (df.loc[valid_mask, 'receiveddate'] - df.loc[valid_mask, 'birth_date']).dt.days / 365.25
    ).round(1)

# Print summary of age calculation
age_count = df['age_at_nom'].notna().sum()
print(f"Successfully calculated age for {age_count} nominees ({age_count/len(df):.1%} of dataset)")
if age_count > 0:
    print(f"Age statistics: min={df['age_at_nom'].min()}, max={df['age_at_nom'].max()}, avg={df['age_at_nom'].mean():.1f}")

Successfully calculated age for 1213 nominees (60.3% of dataset)
Age statistics: min=32.8, max=230.1, avg=50.4


## Creating categorical variable column for seat level

In [ ]:
# identify seat level
df["seat_level"] = (
    df["description"] # we do have court_type columns, but for this coarse an analysis, it's simpler and accurate-enough to look through the nomination description
      .str.lower()
      .str.extract(r"(supreme|circuit|district|(?<=\s)tax|international|appeals)") # insists on whitespace before "tax" so we can tell "tax" or "taxation" etc. are a standalone word
      .fillna("other")
)

In [ ]:
seat_counts = df["seat_level"].value_counts()
print(f"\nDistribution of seat levels (total {len(df)} records):")
for seat, count in seat_counts.items():
    print(f"- {seat}: {count} ({count/len(df):.1%})")

# Show some examples for each seat level for validation
print("\nExample descriptions for each seat level:")
for seat_type in df["seat_level"].unique():
    examples = df[df["seat_level"] == seat_type]["description"].sample(min(2, df[df["seat_level"] == seat_type].shape[0]))
    print(f"\n{seat_type.upper()} examples:")
    for ex in examples:
        print(f"  • {ex}")


Distribution of seat levels (total 2011 records):
- district: 1615 (80.3%)
- circuit: 278 (13.8%)
- other: 37 (1.8%)
- tax: 36 (1.8%)
- appeals: 24 (1.2%)
- international: 17 (0.8%)
- supreme: 4 (0.2%)

Example descriptions for each seat level:

DISTRICT examples:
  • william g. bassler, of new jersey, to be united states district judge for the district of new jersey vice stanley s. brotman, retired.
  • kirk edward sherriff, of california, to be united states district judge for the eastern district of california, vice ana isabel de alba, elevated.

CIRCUIT examples:
  • richard gary taranto, of maryland, to be united states circuit judge for the federal circuit, vice paul r. michel, retired.
  • rebecca ross haywood, of pennsylvania, to be united states circuit judge for the third circuit, vice marjorie o. rendell, retired.

INTERNATIONAL examples:
  • evan j. wallach, of nevada, to be a judge of the united states court of international trade vice edward d. re, retired.
  • jeanne e.

## Unified vs. divided government

Ideally I'd want to use the database behind voteview to get data on how conservative/liberal/etc. the Congress and Senate were at time of nomination and confirmation.
They have an R package from https://github.com/voteview/Rvoteview which looks pretty promising as a way of getting more granular data than just which party controls each half of the legislature.
But their website has been down every time I've checked in the past week, so I gave up on that idea so far.

Lacking that, next best idea I could think to replace it with would be to populate columns for Senate & Congress party composition, at least enough to populate booleans in columns such as:


| Field Name | Data Type | Description | Example | Source |
|------------|-----------|-------------|----------|---------|
| `nom_is_unified` | Boolean | At time of nomination, President's party holds a majority in both the House and the Senate. | false | Calculated from receiveddate column |
| `nom_is_div_opp_house` | Boolean | At time of nomination, President's party holds a majority in the Senate, but the opposition controls the House. | false | Calculated from receiveddate column |
| `nom_is_div_opp_senate` | Boolean | At time of nomination, President's party holds a majority in the House, but the opposition controls the Senate. | true | Calculated from receiveddate column |
| `nom_is_fully_div` | Boolean | At time of nomination, opposition party controls both the House and the Senate. | false | Calculated from receiveddate column |
| `latestaction_is_unified` | Boolean | At time of latest action, President's party holds a majority in both the House and the Senate. | false | Calculated from latestaction_actiondate column |
| `latestaction_is_div_opp_house` | Boolean | At time of latest action, President's party holds a majority in the Senate, but the opposition controls the House. | false | Calculated from latestaction_actiondate column |
| `latestaction_is_div_opp_senate` | Boolean | At time of latest action, President's party holds a majority in the House, but the opposition controls the Senate. | true | Calculated from latestaction_actiondate column |
| `latestaction_is_fully_div` | Boolean | At time of latest action, opposition party controls both the House and the Senate. | false | Calculated from latestaction_actiondate column |

In [ ]:
from nomination_predictor.congress_party_utils import add_alignment_flags

df = add_alignment_flags(df, "party_of_appointing_president", "receiveddate", "latestaction_actiondate")

# Last check for unhelpfully uninformative columns to delete

In [ ]:
df = drop_unhelpfully_uninformative_columns(df)

Columns with limited unique values:
Error analyzing column 'fed_service_sequence': can only convert an array of size 1 to a Python scalar
Error analyzing column 'fed_service_sequence': can only convert an array of size 1 to a Python scalar
  - isprivileged: 1 unique non-null value 'True' (0.2% of rows) - KEEPING
  - party_of_reappointing_president: 1 unique non-null value 'Republican' (0.1% of rows) - KEEPING

No columns were identified for dropping


# Not-yet-implemented analyses ideas:

In [ ]:
# Partisan mismatch: 1 if president_party != party__who_appointed_predecessor
# this would require an additional step of linking predecessor name to nid, and way of looking up when the judge with that nid had been in service, possibly even needing to be a date-and-location-aware analysis

#party_map = {47: "R", 46: "D", 45: "R", 44: "D", 43: "R", 42: "D", 41: "R"}  # extend list
#df["pres_party"] = df["receiveddate"].apply(lambda d: party_map.get(president_number(d), None))
#df["partisan_mismatch"] = (
#    (df["pres_party"].notna()) &
#    (df["pres_party"] != df["party_of_appointing_president"])
#)

In [ ]:

# skipping this one because I think we'd get more and/or richer info out of it if we had a more-successful linkage of the unconfirmed nomination rows between congress and fjc's data
# #Count prior failed nominations for this seat_id (if column present)

#if "other_nominations_count" not in df.columns and "seat_id" in df.columns:
#    prior_counts = (
#        df.groupby("seat_id").cumcount()  # number seen so far for that seat
#    )
#    df["num_prior_failed_noms"] = prior_counts
#
#display(df.head())
#feature_engineered_df = df.copy()

# Saving interim dataframes

In [ ]:
# Save to interim data
df.to_csv( INTERIM_DATA_DIR /"feature_engineered.csv", index=False)

In [ ]:
# Save extracted tables to interim directory
for name, df in dfs.items():
    if len(df) > 0:  # Only save non-empty DataFrames
        output_path = INTERIM_DATA_DIR / f"{name}.csv"
        df.to_csv(output_path, index=False)
        print(f"Saved {len(df)} records to {output_path}")

Saved 4022 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_judges.csv
Saved 4720 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_federal_judicial_service.csv
Saved 4022 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_demographics.csv
Saved 8040 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_education.csv
Saved 611 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_other_federal_judicial_service.csv
Saved 828 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_other_nominations_recess.csv
Saved 19003 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/fjc_professional_career.csv
Saved 5487 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/cong_nominations.csv
Saved 5517 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/cong_nominees.csv
Saved 2011 records to /home/wsl2ubuntuuser/nomination_predictor/data/interim/cong_